In [0]:
##################################################### Importação do arquivo #############################################################################

url = "https://st-it-cloud-public.s3.amazonaws.com/people-v2_1E6.csv.gz"

from pyspark import SparkFiles
spark.sparkContext.addFile(url)
dfPeople = spark.read.csv("file://"+SparkFiles.get("people-v2_1E6.csv.gz"), header=True, inferSchema= True, sep=';')

dfPeople.printSchema()

##################################################### Higienização ##############################################################################

#Higieniza a coluna documento
from pyspark.sql.functions import regexp_replace, col
dfPeople=dfPeople.withColumn("document",regexp_replace(col("document"), "[\-.(\s+)(\D+)]", ""))

#Extrai quantos dígitos cada valor da coluna documento possui, para identificar se é CPF ou CNPJ
from pyspark.sql.functions import length
dfPeople = dfPeople.withColumn("tamanho", length("document"))

from pyspark.sql.functions import when
dfPeople = dfPeople.withColumn("cpfCnpj", when(dfPeople.tamanho == 11,"CPF")
                                          .otherwise("CNPJ"))

#Remove as vírgulas dos registros de data
from pyspark.sql.functions import regexp_replace, col
dfPeople=dfPeople.withColumn("birthDate",regexp_replace(col("birthDate"), ",", ""))

#Para os casos aonde o formato da data está com 15 dígitos, contendo o nome do dia abreviado, é feita essa extração excluindo o dia abreviado e o espaço.Isso é uma etapa para o tratamento posterior, mais abaixo, para deixar todas as datas no mesmo formato, conforme desejado pela especificação.
from pyspark.sql.functions import length, substring
dfPeople=dfPeople.withColumn("birthDateSubstring", substring('BirthDate',5,12))

from pyspark.sql.functions import regexp_replace, col
dfPeople=dfPeople.withColumn("birthDateSubstring",regexp_replace(col("birthDateSubstring"), "[.(\s+)]", "-"))



#Realiza a formatação para um padrão único, normalizando o dado que está em inúmeros formatos diferentes de data. É criado uma coluna temporária para cada formato a ser tratado e depois juntamos os valores tratados em coluna única através da função COALESCE.
                             

from pyspark.sql import functions as F

dfPeople = dfPeople.withColumn(
            'newDate',
                F.to_date(
                    F.unix_timestamp('birthDate', 'MM-dd-yyyy').cast('timestamp')))

dfPeople = dfPeople.withColumn(
            'newDate2',
                F.to_date(
                    F.unix_timestamp('birthDate', 'dd-MM-yyyy').cast('timestamp')))


dfPeople = dfPeople.withColumn(
            'newDate3',
                F.to_date(
                    F.unix_timestamp('birthDate', 'MMM/dd/yyyy').cast('timestamp')))

dfPeople = dfPeople.withColumn(
            'newDate4',
                F.to_date(
                    F.unix_timestamp('birthDate', 'dd-MMM-yyyy').cast('timestamp')))


dfPeople = dfPeople.withColumn(
            'newDate5',
                F.to_date(
                    F.unix_timestamp('birthDate', 'MM/dd/yyyy').cast('timestamp')))

dfPeople = dfPeople.withColumn(
            'newDate6',
                F.to_date(
                    F.unix_timestamp('birthDate', 'yyyyMMdd').cast('timestamp')))

dfPeople = dfPeople.withColumn(
            'newDate7',
                F.to_date(
                    F.unix_timestamp('birthDateSubstring', 'MMM-dd-yyyy').cast('timestamp')))


                             
from pyspark.sql.functions import coalesce
    
dfPeople=dfPeople.withColumn("birthDate",coalesce(dfPeople.newDate,dfPeople.newDate2, dfPeople.newDate3, dfPeople.newDate4, dfPeople.newDate5, dfPeople.newDate6,dfPeople.newDate7)) 


#As duas colunas com informações trocadas para algumas linhas são a phoneNumber e a JobArea, porém o tratamento desta parte não será desenvolvido devido ao prazo de entrega

#Homogeniza o padrão de valores para a coluna Estado
from pyspark.sql.functions import when
dfPeople=dfPeople.withColumn('state',
 when(dfPeople.state==('Santa Catarina'),regexp_replace(dfPeople.state,'Santa Catarina','SC')) \
   .when(dfPeople.state==('Mato Grosso do Sul'),regexp_replace(dfPeople.state,'Mato Grosso do Sul','MS')) \
   .when(dfPeople.state==('Goiás'),regexp_replace(dfPeople.state,'Goiás','GO')) \
   .when(dfPeople.state==('Mato Grosso'),regexp_replace(dfPeople.state,'Mato Grosso','MT')) \
   .when(dfPeople.state==('Ceará'),regexp_replace(dfPeople.state,'Ceará','CE')) \
   .when(dfPeople.state==('Espírito Santo'),regexp_replace(dfPeople.state,'Espírito Santo','ES')) \
   .when(dfPeople.state==('Piauí'),regexp_replace(dfPeople.state,'Piauí','PI')) \
   .when(dfPeople.state==('Paraná'),regexp_replace(dfPeople.state,'Paraná','PR')) \
   .when(dfPeople.state==('Alagoas'),regexp_replace(dfPeople.state,'Alagoas','AL')) \
   .when(dfPeople.state==('Bahia'),regexp_replace(dfPeople.state,'Bahia','BA')) \
   .when(dfPeople.state==('Roraima'),regexp_replace(dfPeople.state,'Roraima','RR')) \
   .when(dfPeople.state==('Distrito Federal'),regexp_replace(dfPeople.state,'Distrito Federal','DF')) \
   .when(dfPeople.state==('Pernambuco'),regexp_replace(dfPeople.state,'Pernambuco','PE')) \
   .when(dfPeople.state==('Amazonas'),regexp_replace(dfPeople.state,'Amazonas','AM')) \
   .when(dfPeople.state==('Acre'),regexp_replace(dfPeople.state,'Acre','AC')) \
   .when(dfPeople.state==('Rio Grande do Sul'),regexp_replace(dfPeople.state,'Rio Grande do Sul','RS'))\
   .when(dfPeople.state==('Rio Grande do Norte'),regexp_replace(dfPeople.state,'Rio Grande do Norte','RN'))\
   .when(dfPeople.state==('Sergipe'),regexp_replace(dfPeople.state,'Sergipe','SE')) \
   .when(dfPeople.state==('São Paulo'),regexp_replace(dfPeople.state,'São Paulo','SP')) \
   .when(dfPeople.state==('Rio de Janeiro'),regexp_replace(dfPeople.state,'Rio de Janeiro','RJ')) \
   .when(dfPeople.state==('Minas Gerais'),regexp_replace(dfPeople.state,'Minas Gerais','MG')) \
   .when(dfPeople.state==('Rondônia'),regexp_replace(dfPeople.state,'Rondônia','RO')) \
   .when(dfPeople.state==('Tocantins'),regexp_replace(dfPeople.state,'Tocantins','TO')) \
   .when(dfPeople.state==('Maranhão'),regexp_replace(dfPeople.state,'Maranhão','MA')) \
   .when(dfPeople.state==('Paraíba'),regexp_replace(dfPeople.state,'Paraíba','PB')) \
   .when(dfPeople.state==('Pará'),regexp_replace(dfPeople.state,'Pará','PA')) \
   .when(dfPeople.state==('Amapá'),regexp_replace(dfPeople.state,'Amapá','AP')) \
   .otherwise(dfPeople.state))



################################################### REPORTS ###########################################################################################

#Demonstra os 5 PFs com maior totalSpent
from pyspark.sql.functions import col
dfpeopleCpfs=dfPeople.filter(dfPeople.cpfCnpj=='CPF')
dfPeoplePFs=dfpeopleCpfs.groupBy("document").sum("totalSpent")
dfPeoplePFs.orderBy(col(("sum(totalSpent)")).desc()).show(5)

#Gasto médio por estado
import pyspark.sql.functions as F
dfMediaEstado=dfPeople.groupBy("state").agg(F.mean("totalSpent"))
dfMediaEstado.show()

#Gasto médio por área
import pyspark.sql.functions as F
dfMediaJobArea=dfPeople.groupBy("jobArea").agg(F.mean("totalSpent"))
dfMediaJobArea.show()

#PF que gastou menos
from pyspark.sql.functions import col
dfPeoplePFsLess = dfPeoplePFs.filter((col("sum(totalSpent)").isNotNull()))
dfPeoplePFs.orderBy(col(("sum(totalSpent)")).asc()).show(1)

#Quantos nomes e documentos repetidos
from pyspark.sql.functions import col
dfPeopleNomesRep=dfPeople.groupBy("document", "name").count()
dfPeopleNomesRep=dfPeopleNomesRep.withColumnRenamed("count","countRenamed")
dfPeopleNomesRep=dfPeopleNomesRep.withColumn("countInt",dfPeopleNomesRep.countRenamed.cast('integer'))
dfPeopleNomesRepFil = dfPeopleNomesRep.filter(dfPeopleNomesRep.countInt > 1)
dfPeopleNomesRepFil.select(count('document')).show()

#Quantas linhas existem no dataset
dfPeople.select(count("document")).show()


                





################################################# GERAÇÃO DOS ARQUIVOS ##################################################################################
dfPeople=dfPeople.drop("birthDateSubstring")\
                 .drop("newDate")\
                 .drop("newDate2")\
                 .drop("newDate3")\
                 .drop("newDate4")\
                 .drop("newDate5")\
                 .drop("newDate6")\
                 .drop("newDate7")\
                 .drop("tamanho")


dfPeople.write.mode('overwrite').partitionBy("state").parquet("/temp/spark_output/problema2_normalizado")
dfPeople.write.format('csv').option('header',True).mode('overwrite').partitionBy("birthDate").option('sep',';').save('file:///home/tangr/output2.csv')


root
-- document: string (nullable = true)
-- name: string (nullable = true)
-- job: string (nullable = true)
-- jobArea: string (nullable = true)
-- jobType: string (nullable = true)
-- phoneNumber: string (nullable = true)
-- birthDate: string (nullable = true)
-- city: string (nullable = true)
-- state: string (nullable = true)
-- totalSpent: double (nullable = true)

+-----------+------------------+
 document| sum(totalSpent)|
+-----------+------------------+
78155376770|1973.1100000000001|
57913684291| 1889.39|
08174237704| 1870.75|
70795760671|1857.1100000000001|
31307158277| 1854.8|
+-----------+------------------+
only showing top 5 rows

+-----+------------------+
state| avg(totalSpent)|
+-----+------------------+
 SC|501.34561245212547|
 RO|498.23731784934995|
 PI| 499.0765211522321|
 AM| 498.7706629805852|
 RR|500.95639783978345|
 GO|501.29249260830375|
 TO|501.90156334817084|
 MT|500.26461002714404|
 SP|498.69447971507907|
 ES|501.63174747811667|
 PB|499.20039918879166|
 RS| 500.6309834299535|
 MS| 499.5097414101175|
 AL|500.51551942023275|
 MG| 499.5320941292197|
 PA| 499.23084708669|
 BA| 498.7741403086009|
 SE|498.24009925154405|
 PE|502.40577268067875|
 CE|499.30029952678933|
+-----+------------------+
only showing top 20 rows

+------------------+---------------+
 jobArea|avg(totalSpent)|
+------------------+---------------+
+55 (81) 3777-4847| 43.85|
+55 (91) 8074-1482| 244.37|
 (69) 2266-7829| 48.84|
 (93) 1477-7460| 670.78|
 (69) 00873-3524| 862.06|
 (00) 87256-8486| 994.55|
 (42) 75457-9490| 421.28|
+55 (82) 7251-0310| 234.4|
 (90) 7060-8461| 583.81|
 (10) 93704-1423| 782.37|
+55 (69) 5996-0558| 159.23|
+55 (48) 6741-1654| 994.87|
+55 (52) 7788-4333| 318.88|
+55 (66) 4496-0159| 530.06|
 (18) 8851-4313| 903.48|
+55 (63) 5153-1855| 312.83|
 (63) 2918-2453| 132.59|
 (60) 8307-0036| 693.49|
+55 (04) 9995-7226| 762.74|
+55 (53) 4440-7248| 104.68|
+------------------+---------------+
only showing top 20 rows

+-----------+---------------+
 document|sum(totalSpent)|
+-----------+---------------+
32737615984| null|
+-----------+---------------+
only showing top 1 row

+---------------+
count(document)|
+---------------+
 0|
+---------------+

+---------------+
count(document)|
+---------------+
 1000000|
+---------------+